In [33]:
import pandas as pd

In [34]:
brm = pd.read_csv("../generated/brm_cleaned.csv")
brm_weather = pd.read_csv("../generated/brm_hourly_weather.csv")

frb = pd.read_csv("../generated/frb_cleaned.csv")
frb_weather = pd.read_csv("../generated/frb_hourly_weather.csv")

brm_hourly_driven = pd.read_csv("../generated/brm_hourly_driven.csv")
frb_hourly_driven = pd.read_csv("../generated/frb_hourly_driven.csv")


In [48]:
def count_unique_per_hour(frame):
    frame["datetime"] = pd.to_datetime(frame["time"], format="%H:%M:%S")
    frame["hour"] = frame["datetime"].map(lambda x: x.hour)
    frame = frame.groupby(["day", "hour"]).agg({"b_number":"nunique"}).reset_index()
    frame = frame.rename(columns={"b_number":"amount_bikes"})
    return frame

def left_merge(first, second):
    first["day"] = pd.to_datetime(first["day"])
    second["day"] = pd.to_datetime(second["day"])
    return first.merge(how="left", left_on=["day", "hour"], right_on=["day","hour"], right=second)


def add_weekday(frame):
    frame["day"] = pd.to_datetime(frame["day"])
    frame["weekday"] = frame["day"].map(lambda x: x.weekday())
    return frame

Merging weather data with trip duration data

In [52]:
brm_with_weather = left_merge(brm_hourly_driven, brm_weather)
frb_with_weather = left_merge(frb_hourly_driven, frb_weather)

brm_with_weather

,day,hour,trip_duration,precipitation,temperature
0,2019-01-20,0,0 days 00:21:00.000000000,0.0,-3.8
1,2019-01-20,1,0 days 00:34:00.000000000,0.0,-3.2
2,2019-01-20,2,0 days 00:32:00.000000000,0.0,-3.6
3,2019-01-20,3,0 days 00:18:00.000000000,0.0,-4.5
4,2019-01-20,5,0 days 00:23:00.000000000,0.0,-4.9
...,...,...,...,...,...
8304,2020-01-20,19,0 days 03:49:00.000000000,0.0,4.4
8305,2020-01-20,20,0 days 04:22:00.000000000,0.0,4.7
8306,2020-01-20,21,0 days 01:29:00.000000000,0.0,4.7
8307,2020-01-20,22,0 days 01:33:00.000000000,0.0,4.7


Calculate amount of used bikes per hour and merge with weather and duration data

In [51]:
brm_hourly_bikes = count_unique_per_hour(brm)
frb_hourly_bikes = count_unique_per_hour(frb)

frb_hourly = left_merge(frb_with_weather, frb_hourly_bikes)
brm_hourly = left_merge(brm_with_weather, brm_hourly_bikes)

frb_hourly = add_weekday(frb_hourly)
brm_hourly = add_weekday(brm_hourly)

brm_hourly.to_csv("../generated/brm_hourly.csv")
frb_hourly.to_csv("../generated/frb_hourly.csv")

brm_hourly

,day,hour,trip_duration,precipitation,temperature,amount_bikes,weekday
0,2019-01-20,0,0 days 00:21:00.000000000,0.0,-3.8,3,6
1,2019-01-20,1,0 days 00:34:00.000000000,0.0,-3.2,3,6
2,2019-01-20,2,0 days 00:32:00.000000000,0.0,-3.6,2,6
3,2019-01-20,3,0 days 00:18:00.000000000,0.0,-4.5,2,6
4,2019-01-20,5,0 days 00:23:00.000000000,0.0,-4.9,1,6
...,...,...,...,...,...,...,...
8304,2020-01-20,19,0 days 03:49:00.000000000,0.0,4.4,19,0
8305,2020-01-20,20,0 days 04:22:00.000000000,0.0,4.7,19,0
8306,2020-01-20,21,0 days 01:29:00.000000000,0.0,4.7,7,0
8307,2020-01-20,22,0 days 01:33:00.000000000,0.0,4.7,9,0
